In [1]:
from google.colab import drive
drive.mount('/content/drive/')
from pandas import DataFrame, Series
import pandas as pd
import glob
import re
from tqdm import tqdm
import torchaudio
import numpy as np
import itertools
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import pickle
import os
import glob
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input 
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from keras.models import load_model

ModuleNotFoundError: No module named 'google.colab'

# 1) Make CSV For Spliting Train, Test Data






In [ ]:
techniques = ['belt','vibrato','vocal_fry', 'straight'] # ['breathy','lip_trill','fast_forte','fast_piano', 'forte','inhaled',  'messa','pp','slow_forte','slow_piano','spoken','trill']
train_singers = ['f1','f3','f4','f5','f6','f7','f9',"m1","m2","m4","m6","m7","m8","m9","m11"]
test_singers = ['f2','f8',"m3","m5","m10"]

train_df = pd.DataFrame(columns = {'path','singer','technique', 'length'})
test_df = pd.DataFrame(columns = {'path','singer','technique', 'length'})

path = "/content/drive/MyDrive/Startup_Hackathon_Jarvis/data_by_technique/"

for technique in techniques:

  file_list =  glob.glob(path + technique + "/*.wav")

  for i in file_list:
    singer = re.findall(r'[mf][0-9]', i)
    sig, sr = torchaudio.load(i)
    length = sig.shape[1]/sr

    if not len(singer): continue

    if singer[0] in test_singers:
      test_df = test_df.append({'path': i,'singer': singer[0],'technique': technique,'length' : length}, ignore_index = True)
    
    elif singer[0] in train_singers:
      train_df = train_df.append({'path': i,'singer': singer[0],'technique': technique,'length' : length}, ignore_index = True)

# train_df.to_csv("/content/drive/MyDrive/Startup_Hackathon_Jarvis/train_df.csv")
# test_df.to_csv("/content/drive/MyDrive/Startup_Hackathon_Jarvis/test_df.csv")

# 2) Saving File & read File


In [ ]:
def read_df(name):
  return pd.read_csv(f"/content/drive/MyDrive/Startup_Hackathon_Jarvis/{name}.csv")

In [ ]:
def save_csv(df, name, folder_name = "pkl_data"):
  tmpdf = pd.DataFrame(df)
  tmpdf.to_csv(f"/content/drive/MyDrive/Startup_Hackathon_Jarvis/{folder_name}/{name}.csv")

In [ ]:
def read_pickle(name, folder_name = "pkl_data"):
  return pd.read_pickle(f"/content/drive/MyDrive/Startup_Hackathon_Jarvis/{folder_name}/{name}.pkl") 

# 3) Show Graphs

### # waveforms

In [ ]:
def show_wave(name, data):
  plt.figure(figsize=(16, 2))
  plt.title(name)
  plt.plot(data)
  plt.ylim((-0.1, 0.1))
  plt.show()



### # spectrograms

In [ ]:
def show_spec(data, name, y_axis, format, x_axis = 'time'):
  librosa.display.specshow(data, sr = 44100, x_axis = x_axis, y_axis = y_axis)
  plt.colorbar(format = format)
  plt.title(name)
  plt.tight_layout()
  plt.show() 


# 4) Data Augmentaion

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

def make_data(df, name, sec = 0.4, save = True):
  with tf.device('/GPU:0'):
    audio = []
    label = []
    speed = [1.0, 1.5, 2.0]
    for i in tqdm(range(len(df))) :
      x, sr = librosa.load(os.path.join('.', df["path"].iloc[i]), sr = 44100)
      if df['technique'].iloc[i] not in ['vibrato','straight']:
        j = 2
        while (j+1)*sec < (df['length'].iloc[i]):
          audio.append(x[int(j*sec*44100) : int((j+1)*sec*44100)]) # 0.4초씩 nonoverlapping하게 자르기
          label.append(df["technique"].iloc[i])
          j += 1

      elif df['technique'].iloc[i] == "straight":
        j = 3
        while (j+1)*sec < (df['length'].iloc[i]-2):
          if j%2 == 0:
            j+=1
            continue
          audio.append(x[int(j*sec*44100) : int((j+1)*sec*44100)])
          label.append(df["technique"].iloc[i])
          j += 1
        
      elif df['technique'].iloc[i] == "vibrato":
        tmp = librosa.effects.time_stretch(x, rate = speed[i%3])
        print(tmp)
        k = 3
        while (k+1)*sec < (len(tmp)/44100-2):
          audio.append(tmp[int(k*sec*44100): int((k+1)*sec*44100)]) # 0.4초씩 nonoverlapping하게 자르기
          label.append(df["technique"].iloc[i])
          k += 1


    if save:
      save_csv({"audio":audio, 'label':label}, f"{name}_data")

    return (audio, label)

train_df = pd.read_csv("/content/drive/MyDrive/Startup_Hackathon_Jarvis/train_df.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Startup_Hackathon_Jarvis/test_df.csv")
train_audio, train_label = make_data(train_df, name = "train")
test_audio, test_label = make_data(test_df, name = "test")

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Startup_Hackathon_Jarvis/train_df.csv")
audio = []
label = []
speed = [1.0, 1.5, 2.0]
sec = 0.4
save = True
name = "train"
folder_name = "pkl_data"
columns = ['audio', 'label']
for i in tqdm(range(len(df))) :
  x, sr = librosa.load(os.path.join('.', df["path"].iloc[i]), sr = 44100)
  if df['technique'].iloc[i] not in ['vibrato','straight']:
    j = 2
    while (j+1)*sec < (df['length'].iloc[i]):
      audio.append(x[int(j*sec*44100) : int((j+1)*sec*44100)]) # 0.4초씩 nonoverlapping하게 자르기
      label.append(df["technique"].iloc[i])
      j += 1

  elif df['technique'].iloc[i] == "straight":
    j = 3
    while (j+1)*sec < (df['length'].iloc[i]-2):
      if j%2 == 0:
        j+=1
        continue
      audio.append(x[int(j*sec*44100) : int((j+1)*sec*44100)])
      label.append(df["technique"].iloc[i])
      j += 1
    
  elif df['technique'].iloc[i] == "vibrato": # ??
    tmp = librosa.effects.time_stretch(x, rate = speed[i%3])
    k = 3
    while (k+1)*sec < ((len(tmp)/44100)-2):
      audio.append(tmp[int(k*sec*44100): int((k+1)*sec*44100)]) # 0.4초씩 nonoverlapping하게 자르기
      label.append(df["technique"].iloc[i])
      k += 1

tmpdf = pd.DataFrame({"audio" : audio, "label" : label})
tmpdf.to_csv(f"/content/drive/MyDrive/Startup_Hackathon_Jarvis/pkl_data/{name}.csv", columns = ['audio', 'label'], mode = 'a')

In [ ]:
def augmentation(audio, label, name, save = True):
  with tf.device('/GPU:0'):
    audio_pitch_shift = []
    for i in tqdm(range(len(audio))):
      audio_pitch_shift.append(librosa.effects.pitch_shift(audio[i], 44100, 4))

    noise = np.random.randn(len(audio[0]))
    noise = (noise - min(noise)) / (max(noise) - min(noise))
    audio_noise = audio + 0.1 * noise
    # Cast back to same data type
    audio_noise = audio_noise.astype(type(audio[0]))


    audio = np.concatenate([audio, audio_pitch_shift, audio_noise],  axis=0) 
    label = np.concatenate([label, label, label]) 

    scaler = StandardScaler()
    audio = scaler.fit_transform(audio)

    tmp = []
    for i in tqdm(range(len(audio))):
      tmp.append([audio[i], label[i]])

    if save:
      save_pickle(tmp, f"augmented_{name}_data")

    return (audio, label)

In [ ]:
def make_mel(audio, label, name, save = True):
    with tf.device('/GPU:0'):
      audio_melspectrogram = []
      for x in audio :
        ret = librosa.feature.melspectrogram(y = x, sr = 44100)
        audio_melspectrogram.append(ret)

      amp_db = []
      for i in range(len(audio_melspectrogram)) :
        amp = librosa.amplitude_to_db(np.abs(audio_melspectrogram[i])) ## np.abs?
        amp_db.append(amp)

      features = []
      for i in range(len(amp_db)) :
        features.append([amp_db[i], label[i]])
    
      if save:
        save_pickle(features, f"mel_{name}_data")

      return pd.DataFrame(features, columns = ['audio', 'label'])
train_mel = make_mel(train_audio, train_label, name = "nonaugmented_train", save = True)
test_mel = make_mel(test_audio, test_label, name = "nonaugmented_test", save = True)

In [ ]:
def make_mfcc(audio, label, name = "", save = True):
  with tf.device('/GPU:0'):
      audio_mfcc = []
      for x in audio:
        ret = librosa.feature.mfcc(y=x, sr=44100, n_mfcc=20)
        audio_mfcc.append(ret)

      amp_db = []
      for i in range(len(audio_mfcc)) :
        amp = librosa.amplitude_to_db(np.abs(audio_mfcc[i])) 
        amp_db.append(amp)

      features = []
      for i in range(len(amp_db)) :
        features.append([amp_db[i], label[i]])
    
      if save:
        save_pickle(features, f"mfcc_{name}_data")

      return pd.DataFrame(features, columns = ['audio', 'label'])

train_mfcc = make_mfcc(train_audio, train_label, name = "nonaugmented_train", save = True)
test_mfcc = make_mfcc(test_audio, test_label, name = "nonaugmented_test", save = True)

In [ ]:
# https://hyongdoc.tistory.com/401
# n_fft가 win_length보다 클 경우, 큰 구간은 모두 zero padding으로 채우게 됩니다. 이유는 설명드린바와 같이, 주파수 해상도를 높이기 위해서입니다.
def make_stft(audio, label, name, save = True):
    with tf.device('/GPU:0'):
      audio_stft = []
      for x in audio:
        ret = librosa.stft(x, n_fft=2048, win_length = 2048, hop_length=512)
        audio_stft.append(ret)

      amp_db = []
      for i in range(len(audio_stft)) :
        amp = librosa.amplitude_to_db(np.abs(audio_stft[i]))
        amp_db.append(amp)

      features = []
      for i in range(len(amp_db)) :
        features.append([amp_db[i], label[i]])

      if save:
         save_pickle(features, f"stft_{name}_data")

      return pd.DataFrame(features, columns = ['audio', 'label'])

train_stft = make_stft(train_audio, train_label, name = "nonaugmented_train", save = True)
test_stft = make_stft(test_audio, test_label, name = "nonaugmented_test", save = True)

In [ ]:
# (2048, 512, 128)
# Investigating Time-Frequency Representations for Audio Feature Extraction in Singing Technique Classification
# 이 논문에서 최고성능을 낸 데이터타입이라 구현해봄
def make_multi_resolution_spectrogram(audio, label, name, save = True):
  with tf.device('/GPU:0'):
      audio_multi = []
      for x in audio:
        stft1 = librosa.stft(x, n_fft=2048, win_length = 2048, hop_length=512)
        stft2 = librosa.stft(x, n_fft=2048, win_length = 512, hop_length=512)
        stft3 = librosa.stft(x, n_fft=2048, win_length = 128, hop_length=512)
        audio_multi.append([stft1, stft2, stft3])

      amp_db = []
      for i in range(len(audio_multi)) :
        amp1 = librosa.amplitude_to_db(np.abs(audio_multi[i][0])) 
        amp2 = librosa.amplitude_to_db(np.abs(audio_multi[i][1]))
        amp3 = librosa.amplitude_to_db(np.abs(audio_multi[i][2]))
        amp_db.append([amp1, amp2, amp3])

      features = []
      for i in range(len(amp_db)) :
        features.append([amp_db[i], label[i]])

      if save:
         save_pickle(features, f"multi_{name}_data")

      return pd.DataFrame(features, columns = ['audio', 'label'])
train_multi = make_multi_resolution_spectrogram(train_audio, train_label, name = "nonaugmented_train", save = True)
test_multi = make_multi_resolution_spectrogram(test_audio, test_label, name = "nonaugmented_test", save = True)


# 5) Data Loader

In [ ]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import random

def train_test(train_features, test_features, shuffle = True, wave_flag=False, labelEncoder = True):
  x_train = np.array(train_features.audio.tolist())
  y_train = np.array(train_features.label.tolist())
  x_test = np.array(test_features.audio.tolist())
  y_test = np.array(test_features.label.tolist())

  if labelEncoder:
    le = LabelEncoder()
    y_train = to_categorical(le.fit_transform(y_train))
    y_test = to_categorical(le.transform(y_test))
    print(le.classes_)

  
  if shuffle: 
    tmp = list(zip(x_train, y_train))
    random.shuffle(tmp)
    x_train, y_train = zip(*tmp)

    tmp1 = list(zip(x_test, y_test))
    random.shuffle(tmp1)
    x_test, y_test = zip(*tmp1)
    x_train, y_train, x_test, y_test = np.array(x_train,dtype=np.float32), np.array(y_train,dtype=np.float32), np.array(x_test,dtype=np.float32), np.array(y_test,dtype=np.float32)
  
  if wave_flag:
    print(x_train.shape)
    x_train = tf.reshape(x_train.astype(float), [-1, x_train.shape[1], 1]) ### 확인
    x_test = tf.reshape(x_test.astype(float), [-1,  x_train.shape[1], 1])
    return x_train, y_train, x_test, y_test, le
  
  n_channels = 1

  with tf.device('/gpu:0'):
    x_train = tf.reshape(x_train, [-1, x_train.shape[1], x_train.shape[2], n_channels])
    x_test = tf.reshape(x_test, [-1,  x_test.shape[1], x_test.shape[2], n_channels])


  return x_train, y_train, x_test, y_test, le

In [ ]:
# x_train, y_train, x_test, y_test, le = train_test(train_mfcc, test_mfcc)

['belt' 'vibrato']


# 6) Training

### # CNN for Waveform(1d array)
+ VocalSet 논문 모델 구현


In [ ]:
# (None, 127, 86, 16) 
def make_waveform_model(input):
  model = keras.Sequential()

  model.add(layers.Conv1D(input_shape=(44100, 1), filters=16, kernel_size=128, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling1D(pool_size=32, strides=8)) # 64
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv1D(kernel_size=64, filters=8, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling1D(pool_size=32, strides=8)) # 64
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv1D(kernel_size=256, filters=32, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling1D(pool_size=32, strides=8)) # 64
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv1D(kernel_size=16, filters=32, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling1D(pool_size=2, strides=2)) # 64
  model.add(layers.Dropout(0.3))

  # model.add(layers.GlobalAveragePooling1D())
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units=32, activation='relu', kernel_regularizer=regularizers.L2(1e-3)))
  model.add(layers.Dropout(0.3))
  model.add(tf.keras.layers.Dense(units=4, activation='softmax', kernel_regularizer=regularizers.L2(1e-3)))

  model.summary()
  return model

tmp = make_waveform_model(1)

training_epochs = 1 # 100
num_batch_size = 64
learning_rate = 1e-3
# opt = keras.optimizers.RMSprop(learning_rate=learning_rate) ## SGD, optimizer 바꿔보기

audio_model = make_waveform_model(audio_x_train)
# keras.optimizers.RMSprop(learning_rate=learning_rate)
# audio_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
audio_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=learning_rate), metrics=tf.keras.metrics.TopKCategoricalAccuracy(k=2, name='top_k_categorical_accuracy'))
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
audio_model.fit(audio_x_train, audio_y_train, batch_size = num_batch_size, epochs = training_epochs , validation_data = (audio_x_test, audio_y_test) , callbacks = [early])
results = audio_model.evaluate(audio_x_test, audio_y_test, batch_size=64)
print('test loss, top2 acc :', results)
tmp = audio_model.predict(audio_x_test)

### # CNN for STFT, Mel, MFCC(2d array)
+ [Investigating Time-Frequency Representations for Audio Feature Extraction in Singing Technique Classification] 논문 모델 구현

In [ ]:
# (None, 127, 86, 16) 
def make_model(input):
  model = keras.Sequential()

  model.add(layers.Conv2D(input_shape=(input[0].shape[0], input[0].shape[1], 1), filters=16, kernel_size=(1,2), activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=2))
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(kernel_size=(1,8), filters=32, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=2))
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(kernel_size=(2,1), filters=64, activation='relu')) # 64
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=3))
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(kernel_size=(8,1), filters=128, activation='relu')) # 128
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=2)) ##
  model.add(layers.Dropout(0.3)) #3

  # model.add(layers.GlobalAveragePooling2D())
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units=64)) # 128
  model.add(tf.keras.layers.Dense(units=22))
  model.add(tf.keras.layers.Dense(units=4, activation='softmax'))

  model.summary()
  return model

training_epochs = 100
num_batch_size = 64
learning_rate = 1e-4

In [ ]:
tmp_model = make_model(x_train)
tmp_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
tmp_model.fit(x_train, y_train, batch_size = num_batch_size, epochs = training_epochs , validation_data = (x_test, y_test) , callbacks = [early])
results = tmp_model.evaluate(x_test, y_test, batch_size=64)
print('test loss, test acc:', results)
tmp_model.save("/content/drive/MyDrive/Startup_Hackathon_Jarvis/model_weight/AWS_test_model.h5")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 20, 34, 16)        48        
                                                                 
 batch_normalization_4 (Batc  (None, 20, 34, 16)       64        
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 10, 17, 16)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 10, 17, 16)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 17, 128)        16512     
                                                                 
 batch_normalization_5 (Batc  (None, 3, 17, 128)      

### # CNN for Multi(3d array)


### # resnet + dense(transfer learning) 

In [ ]:
# The input must have 3 channels; Received `input_shape=(128, 259, 1)`
base_model = tf.keras.applications.ResNet50(
    include_top=False, # 이게 핵심이래
    weights='imagenet',
    input_tensor=None,
    input_shape=(x_train[0].shape[0],x_train[0].shape[1],x_train[0].shape[2]),
    pooling='avg',
)
inputs = Input(shape=(x_train[0].shape[0],x_train[0].shape[1],x_train[0].shape[2]))
x = base_model(inputs, training = False)
x = Flatten()(x)
outputs = Dense(5, activation = 'softmax')(x)
model_res = tf.keras.Model(inputs, outputs)

### # resnet + dense


In [ ]:
base_model = tf.keras.applications.ResNet50(
    include_top=True, # 이게 핵심이래
    weights=None,
    input_tensor=None,
    input_shape=(x_train[0].shape[0],x_train[0].shape[1],x_train[0].shape[2]),
    pooling='avg',
)
inputs = Input(shape=(x_train[0].shape[0],x_train[0].shape[1],x_train[0].shape[2]))
x = base_model(inputs, training = False)
x = Flatten()(x)
outputs = Dense(5, activation = 'softmax')(x)


In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
# model_res.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = 'accuracy')
model_res = tf.keras.Model(inputs, outputs)
model_res.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate= 0.001), loss='categorical_crossentropy', metrics = 'accuracy')
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# train the model on the new data for a few epochs
model_res.fit(x_train, y_train, batch_size = 128, epochs = 20, validation_data = (x_test, y_test), callbacks = [early])

### # resnet

In [ ]:
res_model = tf.keras.applications.ResNet50(
    include_top=True, 
    weights=None,
    input_tensor=None,
    input_shape=(x_train[0].shape[0],x_train[0].shape[1],x_train[0].shape[2]),
    pooling='avg',
    classes = 5
)
res_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.001), loss='categorical_crossentropy', metrics = 'accuracy')
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
# train the model on the new data for a few epochs
res_model.fit(x_train, y_train, batch_size = 128, epochs = 100, validation_data = (x_test, y_test), callbacks = [early])

### ts

In [ ]:
train = read_pickle(name = "")
test = read_pickle(name = "")
audio_x_train, audio_y_train, audio_x_test, audio_y_test, le = train_test(train, test, audio_flag = False)